<a href="https://colab.research.google.com/github/suhas-ds/GenAI_LLM/blob/master/Biology_RAG_Langchain_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Retrieval Augmented Generation (RAG)




1.   Create data directory
2.   Upload **ConceptsofBiology1_new.pdf** and **ConceptsofBiology2_new.pdf** in it

[PDF Files](https://github.com/suhas-ds/GenAI_LLM/tree/master/data/pdfs)
---


data/
*   ConceptsofBiology1_new.pdf
*   ConceptsofBiology2_new.pdf










In [1]:
!pip install -q langchain huggingface_hub sentence_transformers faiss-cpu unstructured==0.5.6 chromadb Cython tiktoken unstructured[local-inference]  groq -q
!pip install -q langchain-community
!pip install -q PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 4

In [2]:
!pip show langchain
!pip show langchain-community
!pip show huggingface_hub
!pip show sentence_transformers
!pip show faiss-cpu
!pip show unstructured
!pip show PyPDF2

Name: langchain
Version: 0.2.14
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community
Name: langchain-community
Version: 0.2.12
Summary: Community contributed LangChain integrations.
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, dataclasses-json, langchain, langchain-core, langsmith, numpy, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
Name: huggingface-hub
Version: 0.23.5
Summary: Client library to download and publish models, datasets and other repos on the huggingface.co hub
Home-page: https://github.com/huggingface/huggingface_hub
Author: Huggin

In [3]:
from huggingface_hub import login
login()

In [4]:
import getpass
import os
import pickle
import warnings
warnings.filterwarnings('ignore')
from langchain_community.document_loaders import PyPDFLoader
import textwrap
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
# if "HUGGINGFACEHUB_API_TOKEN" not in os.environ:
#     os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Provide your HUGGINGFACEHUB TOKEN")

# PDF or Text File

In [ ]:
# Create the directory
!mkdir -p /content/data/

# Change to the new directory
# %cd /content/data/

# Verify the current working directory
%pwd
# !wget https://github.com/suhas-ds/GenAI_LLM/blob/master/data/pdfs/ConceptsofBiology1_new.pdf


In [ ]:
# import requests

# url = "https://raw.githubusercontent.com/suhas-ds/GenAI_LLM/master/data/ConceptsofBiology1and2.txt"
# res = requests.get(url)
# with open("ConceptsofBiology.txt", "w") as f:
#   f.write(res.text)

In [ ]:
# Document Loader

# from langchain.document_loaders import TextLoader
# loader = TextLoader('data/ConceptsofBiology.txt')
# documents = loader.load()
# documents

[Document(metadata={'source': './ConceptsofBiology2.txt'}, page_content="CHAPTER 2\nChemistry of Life\nFIGURE 2.1 Foods such as bread, fruit, and cheese are rich sources of biological macromolecules. (credit:\nmodification of work by Bengt Nyman)\nCHAPTER OUTLINE\n2.1 The Building Blocks of Molecules\n2.2 Water\n2.3 Biological Molecules\nINTRODUCTION The elements carbon, hydrogen, nitrogen, oxygen, sulfur, and phosphorus are\nthe key building blocks of the chemicals found in living things. They form the carbohydrates,\nnucleic acids, proteins, and lipids (all of which will be defined later in this chapter) that are the\nfundamental molecular components of all organisms. In this chapter, we will discuss these\nimportant building blocks and learn how the unique properties of the atoms of different elements\naffect their interactions with other atoms to form the molecules of life.\nFood provides an organism with nutrients—the matter it needs to survive. Many of these critical\nnutrients c

In [5]:
# Document Loader

# Use the following code for loading .pdf file
loader = PyPDFLoader("data/ConceptsofBiology1_new.pdf")
# docs = loader.load_and_split()
# loader = PyPDFDirectoryLoader(doc_directory)
documents = loader.load() # will auto split into pages
print(f"Total Pages: {len(documents)}")

Total Pages: 22


In [5]:
# To print redable output
def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [7]:
print(wrap_text_preserve_newlines(str(documents[1])))

page_content='they do not meet the criteria that biologists use to define life.
From its earliest beginnings, biology has wrestled with four questions: What are the shared
properties that make something “alive”? How do those various living things function? When faced
with the remarkable diversity of life, how do we organize the different kinds of organisms so that
we can better understand them? And, finally—what biologists ultimately seek to understand—how
did this diversity arise and how is it continuing? As new organisms are discovered every day,
biologists continue to seek answers to these and other questions.
Properties of Life
All groups of living organisms share several key characteristics or functions: order, sensitivity or
response to stimuli, reproduction, adaptation, growth and development, regulation/homeostasis,
energy processing, and evolution. When viewed together, these eight characteristics serve to
define life.
Order
Organisms are highly organized structures that consi

In [8]:
# Text Splitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [9]:

len(docs)

22

In [10]:
docs[1]

Document(metadata={'source': 'data/ConceptsofBiology1_new.pdf', 'page': 1}, page_content='they do not meet the criteria that biologists use to define life.\nFrom its earliest beginnings, biology has wrestled with four questions: What are the shared\nproperties that make something “alive”? How do those various living things function? When faced\nwith the remarkable diversity of life, how do we organize the different kinds of organisms so that\nwe can better understand them? And, finally—what biologists ultimately seek to understand—how\ndid this diversity arise and how is it continuing? As new organisms are discovered every day,\nbiologists continue to seek answers to these and other questions.\nProperties of Life\nAll groups of living organisms share several key characteristics or functions: order, sensitivity or\nresponse to stimuli, reproduction, adaptation, growth and development, regulation/homeostasis,\nenergy processing, and evolution. When viewed together, these eight characteri

# Embeddings

In [6]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

with open('docs.pkl', 'wb') as f:
    pickle.dump(db, f)

In [13]:
query = "What is the smallest unit of biological structure that meets the functional requirements of living"
docs_test = db.similarity_search(query)

In [14]:
print(wrap_text_preserve_newlines(str(docs_test[0].page_content)))

FIGURE 1.6 A lot of energy is required for a California condor to fly. Chemical energy derived from food is
used to power flight. Californiacondors are an endangered species; scientists have strived to place a wing tag
on each bird to help them identify and locate each individualbird. (credit: Pacific Southwest Region U.S. Fish
and Wildlife)
Evolution
The diversity of life on Earth is a result of mutations, or random changes in hereditary material over time.
These
mutations allow the possibility for organisms to adapt to a changing environment. An organism that evolves
characteristics fit for the environment will have greater reproductive success, subject to the forces of
natural
selection.
Levels of Organization of Living Things
Living things are highly organized and structured, following a hierarchy on a scale from small to large. The
atom is
the smallest and most fundamental unit of matter that retains the properties of an element. It consists of a
nucleus
surrounded by electrons. A

#QA Chain

In [13]:
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub
from langchain_community.llms import HuggingFaceEndpoint

In [16]:
llm=HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.2", temperature=0.1, max_length=512)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


In [17]:
chain = load_qa_chain(llm, chain_type="stuff")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/v0.2/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag
  warn_deprecated(


In [ ]:
# query = "What is the smallest unit of biological structure that meets the functional requirements of living"
# docs = db.similarity_search(query)
# ans = chain.run(input_documents=docs, question=query)
# print(wrap_text_preserve_newlines('\n'.join(line.strip() for line in ans.split('\n')[:2])))

In [20]:
query = "What type of logical thinking that uses related observations to arrive at a general conclusion"
docs = db.similarity_search(query)
ans = chain.run(input_documents=docs, question=query)
print(wrap_text_preserve_newlines('\n'.join(line.strip() for line in ans.split('\n')[:2])))

Inductive reasoning
Explanation: Inductive reasoning is a form of logical thinking that uses related observations to arrive at a
general conclusion. It is common in descriptive science and involves formulating generalizations inferred from
careful observation and the analysis of a large amount of data. Brain studies often work this way, with many
brains being observed while people are doing a task, and the part of the brain that lights up, indicating
activity, being demonstrated to be the part controlling the response to that task.


In [ ]:
def chatbot_response(query):
    """Generate a response based on the user's query."""
    # Perform similarity search
    docs = db.similarity_search(query)

    # Generate an answer
    ans = chain.run(input_documents=docs, question=query)

    # Format and return the answer
    return wrap_text_preserve_newlines('\n'.join(line.strip() for line in ans.split('\n')[:2]))

print("\nChatbot: Hello! Ask me anything or type 'bye' to end the chat.\n")

while True:
    # Get user input
    user_input = input("You: ")

    # Check if user wants to exit
    if user_input.lower() in ['exit', 'quit', 'bye']:
        print("\nChatbot: Goodbye! Have a great day!\n")
        break

    # Get the chatbot's response
    response = chatbot_response(user_input)

    # Print the response with new lines before and after
    print(f"\nChatbot:\n{response}\n")



Chatbot: Hello! Ask me anything or type 'exit' to end the chat.

You: What is biology?

Chatbot:
Biology is the science of life. It is the study of living organisms and their interactions with each other and
their environment. Biology covers a wide range of topics, from the molecular and cellular level to the level
of populations, communities, and ecosystems. It is a diverse and interdisciplinary field that includes many
branches and sub disciplines, such as molecular biology, microbiology, neurobiology, zoology, and botany,
among others. Biology is essential for understanding the natural world and the processes that sustain life on
Earth. It also has practical applications in fields such as medicine, agriculture, and environmental science.

You: What type of logical thinking that uses related observations to arrive at a general conclusion

Chatbot:
Inductive reasoning
Explanation: Inductive reasoning is a form of logical thinking that uses related observations to arrive at a
general 

#Working with Multiple PDFs Files

In [32]:
# !apt-get update
# !apt-get upgrade
# !sudo apt-get install -y poppler-utils --fix-missing
# !apt-get install libmagic poppler tesseract

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [945 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,560 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,135 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports I

In [57]:
# !pip install "langchain-unstructured[local]"
# !pip install unstructured[local-inference]
# !pip install -q unstructured==0.5.6 --user

In [58]:
# !apt-get update && apt-get install -y libmagic-dev

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pac

In [7]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.indexes import VectorstoreIndexCreator

In [8]:
pdf_folder_path = 'data/'
all_files = os.listdir(pdf_folder_path)
all_files

['ConceptsofBiology1_new.pdf', 'ConceptsofBiology2_new.pdf']

In [61]:
# Filter out only PDF files
pdf_files = [fn for fn in all_files if fn.lower().endswith('.pdf')]
print(pdf_files)
# Create a list of UnstructuredPDFLoader instances for each PDF file
loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in pdf_files]
loaders

['ConceptsofBiology2_new.pdf']


In [9]:
loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)]
loaders

In [10]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

In [11]:
index = VectorstoreIndexCreator(
    embedding=HuggingFaceEmbeddings(),
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)).from_loaders(loaders)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [14]:
llm=HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.2", temperature=0.1, max_length=512)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


In [15]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=index.vectorstore.as_retriever(),
                                    input_key="question")

In [19]:
def chatbot_response(query):
    """Generate a response based on the user's query."""

    # Generate an answer
    ans = chain.run(question=query)

    # Format and return the answer
    return wrap_text_preserve_newlines('\n'.join(line.strip() for line in ans.split('\n')[:2]))

print("\nChatbot: Hello! Ask me anything or type 'bye' to end the chat.\n")

while True:
    # Get user input
    user_input = input("You: ")

    # Check if user wants to exit
    if user_input.lower() in ['exit', 'quit', 'bye']:
        print("\nChatbot: Goodbye! Have a great day!\n")
        break

    # Get the chatbot's response
    response = chatbot_response(user_input)

    # Print the response with new lines before and after
    print(f"\nChatbot:\n{response}\n")



Chatbot: Hello! Ask me anything or type 'bye' to end the chat.

You: What is the smallest unit of biological structure that meets the functional requirements of living

Chatbot:
The smallest unit of biological structure that meets the functional requirements of living is the cell.


You: What type of logical thinking that uses related observations to arrive at a general conclusion

Chatbot:
Inductive reasoning. Inductive reasoning involves formulating generalizations inferred from careful
observation and the analysis of a large amount of data. In the context provided, the student's observation of
a warm classroom leads to the question "Why is the classroom so warm?" Several hypotheses are proposed, and
one is selected based on evidence. The selected hypothesis is then tested, and if it is confirmed, a general
conclusion is reached. This is an example of inductive reasoning.

You: What is biology?

Chatbot:
Biology is the science of life. It is the study of living organisms and their i

# Streamlit app

In [ ]:
# Install necessary packages
!pip install -q streamlit pyngrok

# Run the Streamlit app using ngrok
from pyngrok import ngrok
!ngrok config add-authtoken <enter your ngrok token here>
# Terminate open tunnels if present
ngrok.kill()

# Set up a tunnel for Streamlit
public_url = ngrok.connect(port='8505')
print(f"Streamlit app will be available at: {public_url}")

# Run the Streamlit app
!streamlit run app.py

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


PyngrokNgrokHTTPError: ngrok client exception, API returned 400: {"error_code":102,"status_code":400,"msg":"invalid tunnel configuration","details":{"err":"yaml: unmarshal errors:\n  line 1: field port not found in type config.HTTPv2Tunnel"}}


In [ ]:
# with open('app.py', 'w') as f:
#     f.write('app.py')


# Install necessary packages
!pip install -q streamlit pyngrok

# Run the Streamlit app using ngrok
from pyngrok import ngrok
# !ngrok config add-authtoken <enter your ngrok token here>

# Kill any existing ngrok tunnels
ngrok.kill()

# Create a new ngrok tunnel
public_url = ngrok.connect(8501)
print(f"Streamlit app will be available at: {public_url}")

# Run the Streamlit app
!streamlit run app.py

Streamlit app will be available at: NgrokTunnel: "https://1ef0-34-29-224-28.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.29.224.28:8501

  Stopping...
^C
